<a href="https://colab.research.google.com/github/shauryakudiyal/text-summarization-using-fine-tuned-bart/blob/main/speech_to_text_summarization_using_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download YouTube Video's Audio

In [ ]:
! pip install pytube -q 

In [ ]:
from pytube import YouTube

In [ ]:
VIDEO_URL = 'https://www.youtube.com/watch?v=DoA-a-g2o4g&t=8s' 

In [ ]:
yt = YouTube(VIDEO_URL)

In [ ]:
yt.streams \
  .filter(only_audio = True, file_extension = 'mp4') \
  .first() \
  .download(filename = 'ytaudio.mp4') \
  


'/content/ytaudio.mp4'

In [ ]:
! ffmpeg -i ytaudio.mp4 -acodec pcm_s16le -ar 16000 ytaudio.wav

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

# English ASR


In [ ]:
!pip install huggingsound -q 

In [ ]:
from huggingsound import SpeechRecognitionModel


In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
device

'cuda'

In [ ]:
model = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-english", device = device)


INFO:huggingsound.speech_recognition.model:Loading model...


OUT OF MEMORY (OOM) error

# Audio Chunking

In [ ]:
import librosa

In [ ]:
input_file = '/content/ytaudio.wav'

In [ ]:
print(librosa.get_samplerate(input_file))


stream = librosa.stream(
    input_file,
    block_length=30,
    frame_length=16000,
    hop_length=16000
)

16000


In [ ]:
import soundfile as sf


In [ ]:
for i,speech in enumerate(stream):
  sf.write(f'{i}.wav', speech, 16000)

In [ ]:
i

4

# Audio Transcription / ASR / Speech to Text

In [ ]:
audio_path =[]
for a in range(i+1):
  audio_path.append(f'/content/{a}.wav') 

In [ ]:
audio_path

['/content/0.wav',
 '/content/1.wav',
 '/content/2.wav',
 '/content/3.wav',
 '/content/4.wav']

In [ ]:
transcriptions = model.transcribe(audio_path)

100%|██████████| 5/5 [00:02<00:00,  1.79it/s]


In [ ]:
full_transcript = """ """ 

In [ ]:
for item in transcriptions:
  full_transcript += ''.join(item['transcription'])

In [ ]:
len(full_transcript)

1393

In [ ]:
print(full_transcript)

 soetimes you ned to get knocked now before you can really figure outfighting its and how you need tho fight itsometies you need to feel the pain and sting of eat to activate the real passion and purpose that got predestined inside of yougod says and jeremia i know the plans i have for youans to prosper you and not to harm you plans to give you hope etar future hear me well on this day this day we you have reached te hilltime and you are deciding on on next jobs next steps careers further education you would rather find purpose than a job or careerpurpose crosses disciplines purpose is an suelement of you it is the reason you're only planet at this particular time in history your very existence is wrapped up in the things you are hurtiful fiel whatever you choose for careerpath remember the struggles along the way are only ment to shape you for your purposewhen god has something for you doesn't matter who stands against itif it submit for yougod will mosomeone is holding you back away 



```
# This is formatted as code
```

**TRANSCRIPT CORRECTION(FOR LINGUISTIC AND SPELLING ERRORS)**

In [ ]:
from transformers import pipeline

fix_spelling = pipeline("text2text-generation",model="oliverguhr/spelling-correction-english-base")

fixed_spelling = fix_spelling(full_transcript,max_length=4569)
print(fixed_spelling)

[{'generated_text': 'Sometimes you need to get knocked now before you can really figure outfighting it and how you need, to fight its sometimes you need too feel the pain and sting of eat to activate the real passion and purpose that got predestined inside of you, God says, and Jeremia. I know the plans I have for you, as to prosper you and not to harm your plans to give you hope of a future. He said that his dream was to keep up his game at highest level everyday and God answered is prayers where by in 2015, they became champions of Uganda National League.'}]


transcript error correction

# Text Summarization 

In [ ]:
from transformers import pipeline

In [ ]:
summarization = pipeline('summarization', model="shauryakudiyal/fine-tuned-bart")

In [ ]:
summarized_text = summarization(full_transcript)

In [ ]:
summarized_text[0]['summary_text']

"On this day this day we you have reached te hilltime and you are deciding on on next jobs next steps careers further education you would rather find purpose than a job or career. Remember the struggles along the way are only ment to shape you for your purpose when god has something for you doesn't matter who stands against itif it submit for you."